In [1]:
from __future__ import annotations
from dataclasses import dataclass, field, asdict
from datetime import datetime, timedelta, time
from typing import List, Optional, Dict, Any
import re
import textwrap
import uuid
import pprint

pp = pprint.PrettyPrinter(indent=2).pprint

# -----------------------
# Models
# -----------------------
@dataclass
class Task:
    id: str
    title: str
    description: str
    due_date: Optional[datetime]
    priority: int
    status: str = "pending"
    created_at: datetime = field(default_factory=datetime.now)

@dataclass
class Email:
    id: str
    sender: str
    subject: str
    body: str
    received_at: datetime = field(default_factory=datetime.now)

@dataclass
class CalendarEvent:
    id: str
    title: str
    start_time: datetime
    end_time: datetime
    related_task_id: Optional[str] = None
    created_at: datetime = field(default_factory=datetime.now)

# -----------------------
# Memory / Storage
# -----------------------
class MemoryStore:
    def __init__(self):
        self.tasks: Dict[str, Task] = {}
        self.emails: Dict[str, Email] = {}
        self.events: Dict[str, CalendarEvent] = {}
        self.preferences: Dict[str, Any] = {
            "work_start": time(9, 0),
            "work_end": time(18, 0),
            "default_meeting_duration_minutes": 60
        }

    def add_task(self, title: str, description: str, due_date: Optional[datetime], priority: int) -> Task:
        tid = str(uuid.uuid4())
        task = Task(id=tid, title=title, description=description, due_date=due_date, priority=priority)
        self.tasks[tid] = task
        return task

    def get_pending_tasks(self) -> List[Task]:
        return [t for t in self.tasks.values() if t.status == "pending"]

    def add_email(self, sender: str, subject: str, body: str, received_at: Optional[datetime] = None) -> Email:
        eid = str(uuid.uuid4())
        email = Email(id=eid, sender=sender, subject=subject, body=body, received_at=(received_at or datetime.now()))
        self.emails[eid] = email
        return email

    def add_event(self, title: str, start_time: datetime, end_time: datetime, related_task_id: Optional[str]) -> CalendarEvent:
        eid = str(uuid.uuid4())
        event = CalendarEvent(id=eid, title=title, start_time=start_time, end_time=end_time, related_task_id=related_task_id)
        self.events[eid] = event
        return event

# -----------------------
# Base Agent
# -----------------------
class BaseAgent:
    def __init__(self, name: str, memory: MemoryStore):
        self.name = name
        self.memory = memory

    def handle(self, input_data: dict, context: dict) -> Any:
        raise NotImplementedError

# -----------------------
# Helper utilities
# -----------------------
def simple_date_parse(text: str, now: Optional[datetime] = None) -> Optional[datetime]:
    now = now or datetime.now()
    lo = text.lower()

    if "today" in lo:
        return now.replace(hour=23, minute=59, second=0, microsecond=0)
    if "tomorrow" in lo:
        dt = now + timedelta(days=1)
        return dt.replace(hour=18, minute=0, second=0, microsecond=0)

    m = re.search(r"by\s+(monday|tuesday|wednesday|thursday|friday|saturday|sunday)", lo)
    if m:
        weekdays = { 'monday':0,'tuesday':1,'wednesday':2,'thursday':3,'friday':4,'saturday':5,'sunday':6 }
        target = weekdays[m.group(1)]
        days_ahead = (target - now.weekday() + 7) % 7
        if days_ahead == 0: days_ahead = 7
        dt = now + timedelta(days=days_ahead)
        return dt.replace(hour=18, minute=0, second=0, microsecond=0)

    m = re.search(r"on\s+(\d{4}-\d{2}-\d{2})", text)
    if m:
        try:
            return datetime.fromisoformat(m.group(1))
        except:
            pass

    m = re.search(r"in\s+(\d+)\s+days", lo)
    if m:
        days = int(m.group(1))
        dt = now + timedelta(days=days)
        return dt.replace(hour=18, minute=0, second=0, microsecond=0)

    return None

def simple_priority_score(text: str) -> int:
    lo = text.lower()
    score = 3
    if any(k in lo for k in ["urgent", "asap", "immediately", "right away", "critical"]): score = 5
    elif any(k in lo for k in ["soon", "quick", "by tomorrow", "by friday", "deadline"]): score = 4
    elif any(k in lo for k in ["whenever", "sometime", "no rush"]): score = 2
    return score

def simple_summarize(text: str, max_sentences: int = 2) -> str:
    text = re.sub(r'\s+', ' ', text.strip())
    sentences = re.split(r'(?<=[\.\?\!])\s+', text)
    good = [s.strip() for s in sentences if len(s.strip()) > 10]
    return " ".join(good[:max_sentences]) if good else text[:200]

# -----------------------
# Agents
# -----------------------
class CommsAgent(BaseAgent):
    def handle(self, input_data: dict, context: dict) -> dict:
        email_body = input_data.get("email_text") or ""
        sentences = re.split(r'(?<=[\.\?\!])\s+', email_body)
        sentences = [s.strip() for s in sentences if s.strip()]

        if not sentences:
            task_desc = email_body.strip()[:200]
        else:
            candidates = [s for s in sentences if re.search(r'\b(by|deadline|due|submit|finish|please)\b', s.lower())]
            task_desc = max(candidates or sentences, key=len)

        due_date = simple_date_parse(email_body)
        priority = simple_priority_score(email_body)

        parsed = {
            "task_description": task_desc,
            "due_date": due_date,
            "priority": priority,
            "summary": simple_summarize(email_body)
        }
        context.setdefault("comms_output", {}).update(parsed)
        return parsed

class TaskAgent(BaseAgent):
    def handle(self, input_data: dict, context: dict) -> Task:
        desc = input_data.get("description")
        title = input_data.get("title") or desc[:50]
        due_date = input_data.get("due_date")
        priority = input_data.get("priority", 3)

        task = self.memory.add_task(title, desc, due_date, priority)
        context["created_task"] = task
        return task

class SchedulerAgent(BaseAgent):
    def handle(self, input_data: dict, context: dict) -> Optional[CalendarEvent]:
        task = context.get("created_task")
        if not task: return None

        prefer_dt = task.due_date or (datetime.now() + timedelta(days=1))
        start_dt = datetime.combine(prefer_dt.date(), self.memory.preferences["work_start"])
        duration = self.memory.preferences["default_meeting_duration_minutes"]
        end_dt = start_dt + timedelta(minutes=duration)

        for ev in self.memory.events.values():
            if not (end_dt <= ev.start_time or start_dt >= ev.end_time):
                start_dt += timedelta(hours=1)
                end_dt = start_dt + timedelta(minutes=duration)

        event = self.memory.add_event(f"Work on: {task.title}", start_dt, end_dt, task.id)
        context["calendar_event"] = event
        return event

class KnowledgeAgent(BaseAgent):
    def handle(self, input_data: dict, context: dict) -> dict:
        summary = simple_summarize(input_data.get("text") or "", max_sentences=3)
        note = {
            "summary": summary,
            "source_title": input_data.get("title", "Untitled"),
            "created_at": datetime.now()
        }
        context.setdefault("knowledge", []).append(note)
        return note

class WorkflowAgent(BaseAgent):
    def handle(self, input_data: dict, context: dict) -> dict:
        pending = self.memory.get_pending_tasks()
        today = datetime.now().date()
        todays_events = [e for e in self.memory.events.values() if e.start_time.date() == today]

        summary = {
            "pending_tasks_count": len(pending),
            "pending_tasks": pending,
            "todays_events_count": len(todays_events),
            "todays_events": todays_events
        }
        context["workflow_summary"] = summary
        return summary

# -----------------------
# Orchestrator
# -----------------------
class Orchestrator:
    def __init__(self, memory: MemoryStore):
        self.memory = memory
        self.agents = {
            "comms": CommsAgent("CommsAgent", memory),
            "task": TaskAgent("TaskAgent", memory),
            "scheduler": SchedulerAgent("SchedulerAgent", memory),
            "knowledge": KnowledgeAgent("KnowledgeAgent", memory),
            "workflow": WorkflowAgent("WorkflowAgent", memory)
        }

    def email_to_task_flow(self, sender: str, subject: str, body: str) -> dict:
        context = {}
        email = self.memory.add_email(sender, subject, body)
        context["email"] = email

        comms_out = self.agents["comms"].handle({"email_text": body}, context)
        task = self.agents["task"].handle({
            "description": comms_out["task_description"],
            "due_date": comms_out.get("due_date"),
            "priority": comms_out.get("priority", 3),
            "title": subject
        }, context)
        event = self.agents["scheduler"].handle({}, context)

        return {"email": email, "comms": comms_out, "task": task, "event": event}

    def daily_summary_flow(self) -> dict:
        context = {}
        summary = self.agents["workflow"].handle({}, context)
        return {
            "pending_tasks_count": summary["pending_tasks_count"],
            "pending_tasks": [asdict(t) for t in summary["pending_tasks"]],
            "todays_events_count": summary["todays_events_count"],
            "todays_events": [asdict(e) for e in summary["todays_events"]]
        }

    def routine_automation_flow(self, routine_name="morning_brief"):
        recent_emails = sorted(self.memory.emails.values(), key=lambda e: e.received_at, reverse=True)[:3]
        email_summaries = []
        for e in recent_emails:
            s = self.agents["comms"].handle({"email_text": e.body}, {})
            email_summaries.append({"subject": e.subject, "summary": s["summary"]})

        pending = sorted(self.memory.get_pending_tasks(), key=lambda t: (-t.priority, t.due_date or datetime.max))[:5]

        return {
            "routine_name": routine_name,
            "email_summaries": email_summaries,
            "top_pending_tasks": [asdict(t) for t in pending]
        }

# -----------------------
# Demo / Main
# -----------------------
def demo_full_run():
    print("\n=== Concierge Agents Demo ===\n")
    memory = MemoryStore()
    orch = Orchestrator(memory)

    email1 = {
        "sender": "manager@example.com",
        "subject": "Monthly report submission",
        "body": "Hi — please prepare and submit the monthly report by tomorrow evening. It should include sales numbers and a short summary."
    }
    print(">> Demo 1: Email → Task → Calendar\n")
    result1 = orch.email_to_task_flow(**email1)
    pp(result1)

    email2 = {
        "sender": "teammate@example.com",
        "subject": "Feature doc review",
        "body": "Can you review the feature design document? Please finish by next Friday."
    }
    print("\n>> Demo 2: Second Email\n")
    result2 = orch.email_to_task_flow(**email2)
    pp(result2)

    doc_text = "This document explains the new release plan. Increase testing coverage, fix bugs, and finalize rollout next week."
    print("\n>> Demo 3: Knowledge Summary\n")
    note = orch.agents["knowledge"].handle({"text": doc_text, "title": "Release Plan"}, {})
    pp(note)

    print("\n>> Demo 4: Routine Automation\n")
    print(orch.routine_automation_flow())

    print("\n>> Demo 5: Daily Summary\n")
    print(orch.daily_summary_flow())

if __name__ == "__main__":
    demo_full_run()



=== Concierge Agents Demo ===

>> Demo 1: Email → Task → Calendar

{ 'comms': { 'due_date': datetime.datetime(2025, 11, 30, 18, 0),
             'priority': 4,
             'summary': 'Hi — please prepare and submit the monthly report by '
                        'tomorrow evening. It should include sales numbers and '
                        'a short summary.',
             'task_description': 'Hi — please prepare and submit the monthly '
                                 'report by tomorrow evening.'},
  'email': Email(id='fe6c5de2-9be6-4aa8-bb76-e1782a99c186',
                 sender='manager@example.com',
                 subject='Monthly report submission',
                 body='Hi — please prepare and submit the monthly report by '
                      'tomorrow evening. It should include sales numbers and a '
                      'short summary.',
                 received_at=datetime.datetime(2025, 11, 29, 14, 6, 30, 378670)),
  'event': CalendarEvent(id='017ebe99-2f4e-416d-